In [2]:
# import Library and some random image data set
import tensorflow as tf
import numpy      as np
import seaborn    as sns 
import pandas     as pd
import os,sys
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
np.random.seed(78); tf.set_random_seed(78)

# get some of the STL data set
from sklearn.preprocessing import OneHotEncoder
from skimage import util 
from skimage.transform import resize
from skimage.io import imread
import warnings
from numpy import inf

from scipy.stats import kurtosis,skew

plt.rcParams.update({'font.size': 10})
import warnings
warnings.filterwarnings("ignore")
import gc
from IPython.display import display, clear_output
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from mpl_toolkits.mplot3d import axes3d
from matplotlib import animation
# %load_ext jupyternotify

# read all of the data (STL 10) https://github.com/mttk/STL10
# plt.rcParams.update({'font.size': 10})
# def read_all_images(path_to_data):
#     """
#     :param path_to_data: the file containing the binary images from the STL-10 dataset
#     :return: an array containing all the images
#     """

#     with open(path_to_data, 'rb') as f:
#         # read whole file in uint8 chunks
#         everything = np.fromfile(f, dtype=np.uint8)

#         # We force the data into 3x96x96 chunks, since the
#         # images are stored in "column-major order", meaning
#         # that "the first 96*96 values are the red channel,
#         # the next 96*96 are green, and the last are blue."
#         # The -1 is since the size of the pictures depends
#         # on the input file, and this way numpy determines
#         # the size on its own.

#         images = np.reshape(everything, (-1, 3, 96, 96))

#         # Now transpose the images into a standard image format
#         # readable by, for example, matplotlib.imshow
#         # You might want to comment this line or reverse the shuffle
#         # if you will use a learning algorithm like CNN, since they like
#         # their channels separated.
#         images = np.transpose(images, (0, 3, 2, 1))
#         return images
# def read_labels(path_to_labels):
#     """
#     :param path_to_labels: path to the binary file containing labels from the STL-10 dataset
#     :return: an array containing the labels
#     """
#     with open(path_to_labels, 'rb') as f:
#         labels = np.fromfile(f, dtype=np.uint8)
#         return labels
# def show_images(data,row=1,col=1):
#     fig=plt.figure(figsize=(10,10))
#     columns = col; rows = row
#     for i in range(1, columns*rows +1):
#         fig.add_subplot(rows, columns, i)
#         plt.imshow(data[i-1])
#     plt.show()

# train_images = read_all_images("../../DataSet/STL10/stl10_binary/train_X.bin") / 255.0
# train_labels = read_labels    ("../../DataSet/STL10/stl10_binary/train_Y.bin")
# test_images  = read_all_images("../../DataSet/STL10/stl10_binary/test_X.bin")  / 255.0
# test_labels  = read_labels    ("../../DataSet/STL10/stl10_binary/test_y.bin")

# label_encoder= OneHotEncoder(sparse=False,categories='auto')
# train_labels = label_encoder.fit_transform(train_labels.reshape((-1,1)))
# test_labels  = label_encoder.fit_transform(test_labels.reshape((-1,1)))

# print(train_images.shape,train_images.max(),train_images.min())
# print(train_labels.shape,train_labels.max(),train_labels.min())
# print(test_images.shape,test_images.max(),test_images.min())
# print(test_labels.shape,test_labels.max(),test_labels.min())

In [4]:
# create the layers and the needed functions
def tf_softmax(x): return tf.nn.softmax(x)
def tf_relu(x):   return tf.nn.relu(x)
def d_tf_relu(x): return tf.cast(tf.greater(x,0),tf.float32)
def tf_iden(x): return x
def d_tf_iden(x): return tf.ones_like(x)

class CNN():

    def __init__(self,k,inc,out, stddev=0.05,which_reg='A',act=tf_relu,d_act=d_tf_relu):
        self.w              = tf.Variable(tf.random_normal([k,k,inc,out],stddev=stddev,seed=2,dtype=tf.float32))
        self.m,self.v       = tf.Variable(tf.zeros_like(self.w)),tf.Variable(tf.zeros_like(self.w))
        self.act,self.d_act = act,d_act
        self.current_case   = which_reg
        
    def getw(self): return self.w
    
    def feedforward(self,input,stride=1,padding='SAME',training_phase=True,std_value=0.0005):
        self.input  = input
        
        if self.current_case == 'B':
            def training_fn():  return tf.nn.dropout(tf.nn.conv2d(input,self.w,strides=[1,stride,stride,1],padding=padding),0.8)
            def  testing_fn():  return tf.nn.conv2d(input,self.w,strides=[1,stride,stride,1],padding=padding) 
            self.layer  = tf.cond(training_phase,true_fn=training_fn,false_fn=testing_fn)
            
        elif self.current_case == 'E':
            def training_fn():  return tf.nn.conv2d(input,self.w,strides=[1,stride,stride,1],padding=padding) 
            def  testing_fn():
                sampled_weight = tf.squeeze(tf.distributions.Normal(loc=self.w, scale=std_value).sample(1))
                return tf.nn.conv2d(input,sampled_weight,strides=[1,stride,stride,1],padding=padding) 
            self.layer  = tf.cond(training_phase,true_fn=training_fn,false_fn=testing_fn)
            
        else: self.layer = tf.nn.conv2d(input,self.w,strides=[1,stride,stride,1],padding=padding) 
        
        self.layerA = self.act(self.layer)
        return self.layer, self.layerA
    
    def backprop(self,gradient,std_value,stride=1,padding='SAME'):
        grad_part_1 = gradient
        grad_part_2 = self.d_act(self.layer)
        grad_part_3 = self.input

        grad_middle = grad_part_1 * grad_part_2
        grad        = tf.nn.conv2d_backprop_filter(input = grad_part_3,filter_sizes = tf.shape(self.w),  out_backprop = grad_middle,strides=[1,stride,stride,1],padding=padding) 
        grad_pass   = tf.nn.conv2d_backprop_input (input_sizes = tf.shape(self.input),filter= self.w,out_backprop = grad_middle,strides=[1,stride,stride,1],padding=padding)
        
        if self.current_case == 'D' or self.current_case == 'E': grad = tf.squeeze(tf.distributions.Normal(loc=grad, scale=std_value).sample(1))
        
        update_w = []
        update_w.append(tf.assign( self.m,self.m*beta1 + (1-beta1) * (grad)   ))
        update_w.append(tf.assign( self.v,self.v*beta2 + (1-beta2) * (grad ** 2)   ))
        m_hat = self.m / (1-beta1) ; v_hat = self.v / (1-beta2)
        adam_middle = m_hat * learning_rate/(tf.sqrt(v_hat) + adam_e)
        
        if self.current_case == 'C' or self.current_case == 'D' or self.current_case == 'E': adam_middle = tf.squeeze(tf.distributions.Normal(loc=adam_middle, scale=std_value).sample(1))
        
        update_w.append(tf.assign(self.w,tf.subtract(self.w,adam_middle  )))
        return grad_pass,grad,update_w
    

class RELU_as_Reg():
    
    def __init__(self,batch,width,channel,regularizer):
        self.weigths = tf.Variable(tf.random_uniform([batch,width,width,channel],minval=0.0,maxval=1.0))
        self.regularizer = regularizer
        self.lamda = 0.00005
    
    def feedforward(self,input):
        self.input  = input
        self.layerA = self.weigths * input
        return self.layerA
    
    def backprop(self,gradient):
        grad = gradient * self.input
        gradient_p = self.w * gradient
        
        # add reg here
        if self.regularizer == 'A': grad = grad + self.lamda * tf.sign(self.w)
        if self.regularizer == 'B': grad = grad + self.lamda * 2.0 * self.w
        if self.regularizer == 'C': grad = grad + self.lamda * (1.0/tf.sqrt(tf.square(self.w)+ 1e-5)) * self.w
        if self.regularizer == 'D': grad = grad + self.lamda * -(2*self.w)/(1 + self.w**2)
        if self.regularizer == 'E': grad = grad + self.lamda * -(1-tf.tanh(self.w) ** 2)
        if self.regularizer == 'F': grad = grad + self.lamda * -(1-tf.tanh(self.w** 2) ** 2) * 2.0 * self.w 
        if self.regularizer == 'G': grad = grad + self.lamda * -(1-tf.tanh(tf.abs(self.w)) ** 2) * tf.sign(self.w)
        if self.regularizer == 'H': grad = grad + self.lamda * -(1-tf.tanh(tf.abs(self.w)** 2) ** 2) * 2.0 * tf.abs(self.w) *  tf.sign(self.w)
        if self.regularizer == 'I': grad = grad + self.lamda * tf.cos(self.w)
        if self.regularizer == 'J': grad = grad + self.lamda * tf.sign(tf.sin(self.w)) * tf.cos(self.w)
        if self.regularizer == 'K': grad = grad + self.lamda * (2)/(self.w + 1e-5)
        if self.regularizer == 'L': grad = grad + self.lamda * (tf.log(self.w**2) + 2.0)
        # add reg here

        gradient_temp = tf.clip_by_value(self.w - gradient_w,clip_value_min=0.0,clip_value_max=1.0)
        update_w = []
        update_w.append(self.w.assign(gradient_temp))
        
        return gradient_p,update_w
        

class tf_batch_norm_layer():
    
    def __init__(self,vector_shape,axis):
        self.moving_mean = tf.Variable(tf.zeros(shape=[1,1,1,vector_shape],dtype=tf.float32))
        self.moving_vari = tf.Variable(tf.zeros(shape=[1,1,1,vector_shape],dtype=tf.float32))
        self.axis        = axis
    def feedforward(self,input,training_phase=True,eps = 1e-8):
        self.input = input
        self.input_size          = self.input.shape
        self.batch,self.h,self.w,self.c = self.input_size[0].value,self.input_size[1].value,self.input_size[2].value,self.input_size[3].value

        # Training Moving Average Mean         
        def training_fn():
            self.mean    = tf.reduce_mean(self.input,axis=self.axis ,keepdims=True)
            self.var     = tf.reduce_mean(tf.square(self.input-self.mean),axis=self.axis,keepdims=True)
            centered_data= (self.input - self.mean)/tf.sqrt(self.var + eps)
            
            update_variable = []
            update_variable.append(tf.assign(self.moving_mean,self.moving_mean*0.9 + 0.1 * self.mean ))
            update_variable.append(tf.assign(self.moving_vari,self.moving_vari*0.9 + 0.1 * self.var  ))
            return centered_data,update_variable
        
        # Testing Moving Average Mean        
        def  testing_fn():
            centered_data   = (self.input - self.moving_mean)/tf.sqrt(self.moving_vari + eps)
            update_variable = []
            update_variable.append(tf.assign(self.moving_mean,self.moving_mean))
            update_variable.append(tf.assign(self.moving_vari,self.moving_vari))
            return centered_data,update_variable
        
        self.output,update_variable = tf.cond(training_phase,true_fn=training_fn,false_fn=testing_fn)
        return self.output,update_variable
    def backprop(self,grad,eps = 1e-8):
        change_parts = 1.0 /(self.batch * self.h * self.w)
        grad_sigma   = tf.reduce_sum( grad *  (self.input-self.mean)     ,axis=self.axis,keepdims=True) * -0.5 * (self.var+eps) ** -1.5
        grad_mean    = tf.reduce_sum( grad *  (-1./tf.sqrt(self.var+eps)),axis=self.axis,keepdims=True) + grad_sigma * change_parts * 2.0 * tf.reduce_sum((self.input-self.mean),axis=self.axis,keepdims=True) * -1
        grad_x       = grad * 1/(tf.sqrt(self.var+eps)) + grad_sigma * change_parts * 2.0 * (self.input-self.mean) + grad_mean * change_parts
        return grad_x